In [1]:
!conda activate cloudspace

In [13]:
import os
import shutil
from PIL import Image  # Don't forget to import Image from PIL

# Specify the paths
source_folder = './image_log/train'
ground_truth_folder = './to_evaluate/existing'
generated_folder = './to_evaluate/generated'

# Create the destination folders if they don't already exist
#os.makedirs(ground_truth_folder, exist_ok=True)
#os.makedirs(generated_folder, exist_ok=True)

countg = 0
countt = 0

# Loop through each file in the source folder
for filename in os.listdir(source_folder):
    file_path = os.path.join(source_folder, filename)

    # Open the image
    image = Image.open(file_path)
    width, height = image.size
        
    # Calculate width of each split panel (assuming 4 equal horizontal panels)
    panel_width = width // 4

    if "samples" in filename:
        # Define bounding boxes for the 4 panels
        panels = {
            "generated_1": (0, 0, panel_width, height),
            "generated_2": (panel_width, 0, 2 * panel_width, height),
            "generated_3": (2 * panel_width, 0, 3 * panel_width, height),
            "generated_4": (3 * panel_width, 0, width, height)
        }
    
        # Crop and save each panel for generated images
        for panel_name, box in panels.items():
            panel_image = image.crop(box)
            save_path = os.path.join(generated_folder, f"generated_{countg}.png")
            panel_image.save(save_path)
            countg += 1
    elif "reconstruction" in filename:
        # Define bounding boxes for the 4 panels for ground truth images
        panels = {
            "true_1": (0, 0, panel_width, height),
            "true_2": (panel_width, 0, 2 * panel_width, height),
            "true_3": (2 * panel_width, 0, 3 * panel_width, height),
            "true_4": (3 * panel_width, 0, width, height)
        }
    
        # Crop and save each panel for ground truth images
        for panel_name, box in panels.items():
            panel_image = image.crop(box)
            save_path = os.path.join(ground_truth_folder, f"true_{countt}.png")
            panel_image.save(save_path)
            countt += 1
            


In [6]:
# example of calculating the frechet inception distance in Keras for cifar10
import numpy as np
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import shuffle
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets.mnist import load_data
from skimage.transform import resize
import os
from PIL import Image

def load_img(folder_path):
    images = list()
    # Loop through each file in the folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        image_array = np.array(Image.open(file_path).convert('RGB'))
        resized_image = resize(image_array, (299, 299, 3), anti_aliasing=True)
        images.append(resized_image)
    return asarray(images)
 
# calculate frechet inception distance
def calculate_fid(model, images1, images2):
	# calculate activations
	act1 = model.predict(images1)
	act2 = model.predict(images2)
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = np.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid
 
# prepare the inception v3 model
model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))

images1=load_img('./to_evaluate/existing')
images2=load_img('./to_evaluate/generated')
print('Loaded', images1.shape, images2.shape)

# convert integer to floating point values
images1 = images1.astype('float32')
images2 = images2.astype('float32')

# pre-process images
images1 = preprocess_input(images1)
images2 = preprocess_input(images2)
# calculate fid
fid = calculate_fid(model, images1, images2)
print('FID: %.3f' % fid)


Loaded (40, 299, 299, 3) (40, 299, 299, 3)
FID: 3.664
